<a href="https://colab.research.google.com/github/alikashcherba/alikashcherba/blob/main/%D1%82%D0%B3_%D0%B1%D0%BE%D1%82_HealthAssistant_of_yours_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Телеграм-бот-помощник по поддержанию здорового образа жизни

@HealthAssistant_of_yours_bot



In [1]:
!pip install aiogram aiosqlite apscheduler langchain requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.2/617.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


In [ ]:
from google.colab import userdata
token = userdata.get('TelegramBot')
GigaChatKey = userdata.get('GigaChatKey')

In [ ]:
import os
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")


In [10]:
from aiogram import Bot, Dispatcher
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram import Router, F
from aiogram.filters import CommandStart, Command
from aiogram.types import Message, KeyboardButton, ReplyKeyboardMarkup, BotCommand
from aiogram.fsm.context import FSMContext
from aiogram.fsm.state import State, StatesGroup
import logging
import asyncio
import aiosqlite
from aiogram import types
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from apscheduler.triggers.interval import IntervalTrigger
from apscheduler.triggers.cron import CronTrigger
from aiogram.types import InlineKeyboardMarkup, InlineKeyboardButton
from aiogram.types import CallbackQuery
from datetime import datetime, timedelta
import locale
from pytz import timezone
import os
import requests
import json
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import base64
import uuid
import re
import aiosqlite



def generate_rquid():
    return str(uuid.uuid4())



def localize_time(utc_time, user_timezone='Europe/Moscow'):
    tz = timezone(user_timezone)
    return utc_time.astimezone(tz)

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Инициализация бота и диспетчера
bot = Bot(token)
dp = Dispatcher(storage=MemoryStorage())
scheduler = AsyncIOScheduler()

start_router = Router()
dp.include_router(start_router)

credentials = f"{client_id}:{client_secret}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()
authorization_header = f"Basic {encoded_credentials}"

# Отключаем предупреждения об использовании небезопасного соединения
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

def get_gigachat_access_token():
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    payload = {'scope': 'GIGACHAT_API_PERS'}
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': generate_rquid(),
        'Authorization': f'Basic {GigaChatKey}'
    }

    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        raise Exception(f"Ошибка при получении токена: {response.text}")

def get_gigachat_response(prompt: str, token: str) -> str:
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    payload = json.dumps({
        "model": "GigaChat",
        "messages": [
            {"role": "user",
            "content": prompt}
        ],
        "temperature": 0.5,
        "top_p": 0.1, #Контроль разнообразия ответов
        "n": 1, #Кол-во возвращаемых ответов
        "stream": False, #Потоковая передача ответа
        "max_tokens": 512, #Максимальное количество токенов в ответе
        "repetition_penalty": 1, #Штраф за повторения
        "update_interval": 0 #Интервал обновления (для потоковой передачи)
})


    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {token}'
    }

    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    if response.status_code == 200:
        return response.json().get("choices")[0].get("message").get("content")
    else:
        raise Exception(f"Ошибка при вызове GigaChat API: {response.text}")

def clean_response(response: str) -> str:
    response = re.sub(r"[*#]+", "", response)
    response = re.sub(r"\s*\n\s*\n\s*", "\n", response.strip())
    return response

# Определение состояний
class UserForm(StatesGroup):
    age = State()
    height = State()
    weight = State()
    goals = State()
    food_preferences = State()
    adding_training = State()
    setting_reminder_time = State()
    setting_reminder_frequency = State()
    waiting_for_time = State()
    waiting_for_message = State()
    select_reminder_type = State()

# Инициализация базы данных
async def init_db():
    async with aiosqlite.connect('workout_diary.db') as db:
        await db.execute('''
            CREATE TABLE IF NOT EXISTS users (
                user_id INTEGER PRIMARY KEY,
                age INTEGER,
                height INTEGER,
                weight INTEGER,
                goals TEXT,
                food_preferences TEXT
            )
        ''')
        await db.execute('''
            CREATE TABLE IF NOT EXISTS trainings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER,
                date TEXT,
                training TEXT,
                FOREIGN KEY (user_id) REFERENCES users (user_id)
            )
        ''')
        await db.execute('''
            CREATE TABLE IF NOT EXISTS jobs (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER NOT NULL,
                job_type TEXT NOT NULL,
                time TEXT NOT NULL,
                frequency TEXT NOT NULL
            )
        ''')
        await db.commit()

async def save_job_to_db(user_id, job_type, time, frequency):
    async with aiosqlite.connect('workout_diary.db') as db:
        await db.execute('''
            INSERT INTO jobs (user_id, job_type, time, frequency) VALUES (?, ?, ?, ?)
        ''', (user_id, job_type, time, frequency))
        await db.commit()

async def load_jobs_from_db():
    async with aiosqlite.connect('workout_diary.db') as db:
        async with db.execute('SELECT user_id, job_type, time, frequency FROM jobs') as cursor:
            return await cursor.fetchall()
"""
async def restore_jobs():
    jobs = await load_jobs_from_db()
    for job in jobs:
        user_id, job_type, time, frequency = job
        await schedule_reminders(user_id, time, frequency)
        """


def create_training_options_keyboard():
    keyboard = InlineKeyboardMarkup(
        inline_keyboard=[
            [
                InlineKeyboardButton(text="Похудение", callback_data="training_goal:loose_weight"),
                InlineKeyboardButton(text="Набор веса", callback_data="training_goal:get_weight"),
                InlineKeyboardButton(text="Поддержание", callback_data="training_goal:keep_balance"),
            ]
        ]
    )
    return keyboard


async def save_user_data(user_id, age, height, weight, goals, food_preferences):
    async with aiosqlite.connect('workout_diary.db') as db:
        await db.execute('''
            INSERT OR REPLACE INTO users (user_id, age, height, weight, goals, food_preferences)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (user_id, age, height, weight, goals, food_preferences))
        await db.commit()
    logger.info(f"Данные пользователя {user_id} сохранены: {age}, {height}, {weight}, {goals}, {food_preferences}")

async def set_bot_commands():
    commands = [
        BotCommand(command="start", description="Старт"),
        BotCommand(command="recommendation", description="Рекомендации по питанию"),
        BotCommand(command="train_recommendations", description="Рекомендации по тренировкам"),
        BotCommand(command="add_training", description="Добавить тренировку"),
        BotCommand(command="see_trainings", description="Посмотреть мои тренировки"),
        BotCommand(command="set_reminder_time", description="Настроить напоминания о питье воды и выполнению зарядки."),
        BotCommand(command="edit_profile", description="Редактировать профиль")
    ]
    await bot.set_my_commands(commands)


@dp.message(Command("start"))
async def cmd_start(message: Message, state: FSMContext):
    user_id = message.from_user.id
    async with aiosqlite.connect('workout_diary.db') as db:
        async with db.execute('SELECT * FROM users WHERE user_id = ?', (user_id,)) as cursor:
            user_data = await cursor.fetchone()
            if user_data:
                await message.answer(
                      "Спасибо за регистрацию! Вам доступен следующий функционал.\n"
                      "Доступные команды:\n"
                      "/start - Начать взаимодействие\n"
                      "/recommendation - Получить рекомендации по питанию\n"
                      "/train_recommendations - Получить рекомендации по тренировкам\n"
                      "/add_training - Добавить тренировку\n"
                      "/see_trainings - Посмотреть мои тренировки\n"
                      "/set_reminder_time - Настроить напоминания\n"
                      "/edit_profile - Редактировать профиль"
                  )
            else:
                await message.answer("Привет! Давайте начнем с регистрации. Сколько вам лет?")
                await state.set_state(UserForm.age)


@dp.message(UserForm.age)
async def process_age(message: Message, state: FSMContext):
    try:
        age = int(message.text)
        if age < 10 or age > 100:
            raise ValueError("Возраст должен быть в диапазоне от 10 до 100 лет.")
        await state.update_data(age=age)

        await message.answer("Введите ваш рост (в см):")
        await state.set_state(UserForm.height)
    except ValueError:
        await message.answer("Пожалуйста, введите корректный возраст (число от 10 до 100).")


@dp.message(UserForm.height)
async def process_height(message: Message, state: FSMContext):
    try:
        height = int(message.text)
        if height < 50 or height > 250:
            raise ValueError("Рост должен быть в диапазоне от 50 до 250 см.")
        await state.update_data(height=height)
        await message.answer("Введите ваш вес (в кг):")
        await state.set_state(UserForm.weight)
    except ValueError:
        await message.answer("Пожалуйста, введите корректный рост (число от 50 до 250).")


@dp.message(UserForm.weight)
async def process_weight(message: Message, state: FSMContext):
    try:
        weight = int(message.text)
        if weight < 20 or weight > 300:
            raise ValueError("Вес должен быть в диапазоне от 20 до 300 кг.")
        await state.update_data(weight=weight)
        keyboard = create_training_options_keyboard()
        await message.answer("Выберите вашу цель:", reply_markup=keyboard)
        await state.set_state(UserForm.goals)
    except ValueError:
        await message.answer("Пожалуйста, введите корректный вес (число от 20 до 300).")

# Обработка выбора цели
@dp.callback_query(F.data.startswith("training_goal:"))
async def process_training_goal(callback_query: CallbackQuery, state: FSMContext):
    current_state = await state.get_state()
    if current_state != UserForm.goals.state:
        await callback_query.answer("Это действие недоступно сейчас.", show_alert=True)
        return
    goal = callback_query.data.split(":")[1]

    goal_mapping = {
        "loose_weight": "Похудение",
        "get_weight": "Набор веса",
        "keep_balance": "Поддержание"
    }
    await state.update_data(goals=goal_mapping.get(goal, "Неизвестная цель"))
    await callback_query.answer()
    await callback_query.message.answer("Каковы ваши предпочтения в еде?")
    await state.set_state(UserForm.food_preferences)



@dp.message(UserForm.food_preferences)
async def process_food_preferences(message: Message, state: FSMContext):
    food_preferences = message.text.strip()
    if len(food_preferences) < 3:
        await message.answer("Предпочтения в еде должны быть описаны минимум в 3 символах. Пожалуйста, уточните.")
        return

    user_data = await state.get_data()
    await save_user_data(
        message.from_user.id,
        user_data['age'],
        user_data['height'],
        user_data['weight'],
        user_data['goals'],
        food_preferences
    )
    await message.answer(
        f"Спасибо за регистрацию!\nВаши данные:\n"
        f"Возраст: {user_data['age']}\n"
        f"Рост: {user_data['height']}\n"
        f"Вес: {user_data['weight']}\n"
        f"Цели: {user_data['goals']}\n"
        f"Предпочтения: {food_preferences}"
    )
    await state.set_state(None)

@dp.message(Command("recommendation"))
async def cmd_recommendations(message: Message):
    try:
        token = get_gigachat_access_token()

        async with aiosqlite.connect('workout_diary.db') as db:
            async with db.execute('SELECT age, height, weight, goals, food_preferences FROM users WHERE user_id = ?', (message.from_user.id,)) as cursor:
                user_data = await cursor.fetchone()
                if user_data:
                    age, height, weight, goals, food_preferences = user_data
                    logger.info(f"Данные пользователя {message.from_user.id} извлечены: {user_data}")
                    prompt = (
                        "Напиши рекомендации по питанию для пользователя со следующими характеристиками: "
                        f"возраст: {age}, рост: {height} см, вес: {weight} кг, цели: {goals}, "
                        f"предпочтения по еде: {food_preferences}."
                    )

                    response = get_gigachat_response(prompt, token)
                    clean_response_text = clean_response(response)
                    await message.answer(f"Ваши персонализированные рекомендации по питанию:\n{clean_response_text}")
                else:
                    logger.warning(f"Пользователь {message.from_user.id} не найден в базе данных.")
                    await message.answer("Сначала пройдите регистрацию, чтобы получить рекомендации.")

    except Exception as e:
        logger.error(f"Ошибка при вызове GigaChat: {e}")
        await message.answer("Произошла ошибка при получении рекомендаций. Попробуйте позже.")

@dp.message(Command("train_recommendations"))
async def cmd_train_recommendations(message: Message):
    try:
        token = get_gigachat_access_token()
        async with aiosqlite.connect('workout_diary.db') as db:
            async with db.execute('SELECT age, height, weight, goals, food_preferences FROM users WHERE user_id = ?', (message.from_user.id,)) as cursor:
                user_data = await cursor.fetchone()
                if user_data:
                    age, height, weight, goals, food_preferences = user_data
                    logger.info(f"Цели пользователя {message.from_user.id} извлечены: {goals}")

                    prompt = (
                        "Напиши рекомендации по тренировкам для пользователя с целью: "
                        f"{goals}. Вот некоторые характеристики пользователя: возраст: {age}, рост: {height} см," +
                        f"вес: {weight} кг. Укажи примерную частоту, виды упражнений и акценты на мышцы."
                    )

                    response = get_gigachat_response(prompt, token)
                    clean_response_text = clean_response(response)
                    await message.answer(f"Ваши персонализированные рекомендации по тренировкам:\n{clean_response_text}")
                else:
                    logger.warning(f"Пользователь {message.from_user.id} не найден в базе данных.")
                    await message.answer("Сначала пройдите регистрацию, чтобы получить рекомендации.")
    except Exception as e:
        logger.error(f"Ошибка при вызове GigaChat: {e}")
        await message.answer("Произошла ошибка при получении рекомендаций. Попробуйте позже.")


@dp.message(Command("add_training"))
async def cmd_add_training(message: Message, state: FSMContext):
    await message.answer("Введите описание вашей тренировки:")
    await state.set_state(UserForm.adding_training)

@dp.message(UserForm.adding_training)
async def process_training(message: Message, state: FSMContext):
    user_id = message.from_user.id
    training_description = message.text
    date = message.date.isoformat()

    async with aiosqlite.connect('workout_diary.db') as db:
        await db.execute('''
            INSERT INTO trainings (user_id, date, training)
            VALUES (?, ?, ?)
        ''', (user_id, date, training_description))
        await db.commit()

    await message.answer("Тренировка успешно добавлена!")
    await state.set_state(None)

@dp.message(Command("see_trainings"))
async def cmd_see_trainings(message: Message):
    user_id = message.from_user.id
    async with aiosqlite.connect('workout_diary.db') as db:
        async with db.execute('SELECT date, training FROM trainings WHERE user_id = ?', (user_id,)) as cursor:
            trainings = await cursor.fetchall()
            if trainings:
                response = "Ваши тренировки:\n"
                for date, training in trainings:
                    parsed_date = datetime.fromisoformat(date)
                    formatted_time = parsed_date.strftime("%d %B %Y, %H:%M")
                    response += f"{formatted_time}: {training}\n"
                await message.answer(response)
            else:
                await message.answer("У вас нет сохраненных тренировок.")


@dp.message(Command("set_reminder_time"))
async def cmd_set_reminder_time(message: Message, state: FSMContext):
    await message.answer("Введите время для напоминания в формате ЧЧ:ММ по UTC (на 3 часа раньше, если вы в Москве). Например, если нужно на 08:00, пишете 05:00")
    await state.set_state(UserForm.waiting_for_time)


@dp.message(UserForm.waiting_for_time)
async def process_time_input(message: Message, state: FSMContext):
    try:
        user_time = datetime.strptime(message.text, "%H:%M").time()
        user_timezone = timezone('Europe/Moscow')
        now = datetime.now(user_timezone)
        reminder_time = datetime.combine(now.date(), user_time, tzinfo=user_timezone)
        await state.update_data(reminder_time=reminder_time.strftime("%H:%M"))

        await message.answer(
            "Выберите частоту напоминаний:",
            reply_markup=get_frequency_keyboard()
        )
        await state.set_state(UserForm.setting_reminder_frequency)
    except ValueError:
        await message.answer("Неверный формат времени! Введите время в формате ЧЧ:ММ (например, 08:00):")


@dp.callback_query(UserForm.setting_reminder_frequency)
async def process_frequency_selection(callback_query: CallbackQuery, state: FSMContext):
    frequency_map = {
        "frequency_daily": "каждый день",
        "frequency_2hours": "каждые 2 часа",
        "frequency_hourly": "каждый час",
    }
    frequency_input = frequency_map.get(callback_query.data)
    if not frequency_input:
        await callback_query.answer("Ошибка: неизвестная частота.")
        return

    # Обновляем данные в FSMContext
    await state.update_data(reminder_frequency=frequency_input)

    await callback_query.message.answer(
        "Что вы хотите, чтобы я напоминал вам? \n1 - Питье воды\n2 - Зарядка",
        reply_markup=None
    )
    await state.set_state(UserForm.select_reminder_type)


@dp.message(UserForm.select_reminder_type)
async def select_reminder_type(message: Message, state: FSMContext):
    reminder_type = message.text.strip()
    if reminder_type not in ["1", "2"]:
        await message.answer("Выберите корректный вариант: 1 - Питье воды, 2 - Зарядка.")
        return

    reminder_text = "Пора выпить воды!" if reminder_type == "1" else "Не забудьте сделать зарядку!"

    await state.update_data(reminder_type=reminder_text)

    user_data = await state.get_data()
    reminder_time = user_data.get('reminder_time')
    reminder_frequency = user_data.get('reminder_frequency')

    await schedule_reminders(message.from_user.id, reminder_time, reminder_frequency, reminder_text)

    await message.answer(f"Напоминания настроены: {reminder_text} в {reminder_time} с частотой {reminder_frequency}.")
    await state.set_state(None)


async def schedule_reminders(user_id: int, reminder_time: str, frequency: str, reminder_text: str):
    now = datetime.now()
    try:
        reminder_time_obj = datetime.strptime(reminder_time, "%H:%M").time()
        reminder_datetime = datetime.combine(now.date(), reminder_time_obj)

        if reminder_datetime < now:
            reminder_datetime += timedelta(days=1)

        if frequency == "каждый день":
            trigger = CronTrigger(hour=reminder_time_obj.hour, minute=reminder_time_obj.minute)
        elif frequency == "каждые 2 часа":
            trigger = IntervalTrigger(hours=2, start_date=reminder_datetime)
        elif frequency == "каждый час":
            trigger = IntervalTrigger(hours=1, start_date=reminder_datetime)
        else:
            raise ValueError("Неверная частота напоминаний.")

        scheduler.add_job(
            send_reminder,
            trigger=trigger,
            args=[user_id, reminder_text],
            id=f"reminder_{user_id}_{frequency}",
            replace_existing=True
        )
        await save_job_to_db(user_id, "reminder", reminder_time, frequency)
    except ValueError as e:
        logger.error(f"Ошибка в настройке напоминания: {e}")


async def send_reminder(user_id: int, reminder_text: str):
    try:
        await bot.send_message(user_id, reminder_text)
    except Exception as e:
        logger.error(f"Ошибка отправки напоминания пользователю {user_id}: {e}")


async def restore_jobs():
    jobs = await load_jobs_from_db()
    for job in jobs:
        user_id, job_type, time, frequency = job
        if job_type == "reminder":
            reminder_text = "Пора выпить воды!"
        else:
            reminder_text = "Не забудьте сделать зарядку!"
        await schedule_reminders(user_id, time, frequency, reminder_text)

# Клавиатура для выбора частоты
def get_frequency_keyboard():
    keyboard = InlineKeyboardMarkup(inline_keyboard=[
        [
            InlineKeyboardButton(text="Каждый час", callback_data="frequency_hourly"),
            InlineKeyboardButton(text="Каждые 2 часа", callback_data="frequency_2hours"),
        ],
        [
            InlineKeyboardButton(text="Каждый день", callback_data="frequency_daily"),
        ],
    ])
    return keyboard


# Команда /edit_profile
@dp.message(Command("edit_profile"))
async def cmd_edit_profile(message: Message, state: FSMContext):
    await message.answer("Введите ваш новый возраст:")
    await state.set_state(UserForm.age)

# Обработчик для изменения возраста
@dp.message(UserForm.age)
async def process_new_age(message: types.Message, state: FSMContext):
    try:
        new_age = int(message.text)
        if new_age < 0 or new_age > 120:
            raise ValueError("Возраст должен быть в пределах от 0 до 120 лет.")
        await state.update_data(age=new_age)
        await message.answer(f"Возраст обновлен: {new_age} лет. Теперь введите ваш новый рост (в см):")
        await state.set_state(UserForm.height)
    except ValueError:
        await message.answer("Пожалуйста, введите корректный возраст.")

# Обработчик для изменения роста
@dp.message(UserForm.height)
async def process_new_height(message: types.Message, state: FSMContext):
    try:
        new_height = int(message.text)
        if new_height < 50 or new_height > 250:
            raise ValueError("Рост должен быть в пределах от 50 до 250 см.")
        await state.update_data(height=new_height)
        await message.answer(f"Рост обновлен: {new_height} см. Теперь введите ваш новый вес (в кг):")
        await state.set_state(UserForm.weight)
    except ValueError:
        await message.answer("Пожалуйста, введите корректный рост.")

# Обработчик для изменения веса
@dp.message(UserForm.weight)
async def process_new_weight(message: types.Message, state: FSMContext):
    try:
        new_weight = float(message.text)
        if new_weight < 30 or new_weight > 300:
            raise ValueError("Вес должен быть в пределах от 30 до 300 кг.")
        await state.update_data(weight=new_weight)
        await message.answer(f"Вес обновлен: {new_weight} кг. Теперь выберите новую цель: Похудение, Набор веса, Поддержание.")
        await state.set_state(UserForm.goals)
    except ValueError:
        await message.answer("Пожалуйста, введите корректный вес.")

# Обработчик для изменения цели
@dp.message(UserForm.goals)
async def process_new_goals(message: types.Message, state: FSMContext):
    goal_mapping = {
        "Похудение": "Похудение",
        "Набор веса": "Набор веса",
        "Поддержание": "Поддержание"
    }

    goal = message.text.strip()
    if goal in goal_mapping:
        await state.update_data(goals=goal)
        await message.answer(f"Цель обновлена на: {goal}. Теперь введите ваши предпочтения в еде:")
        await state.set_state(UserForm.food_preferences)
    else:
        await message.answer("Выберите корректную цель: Похудение, Набор веса, Поддержание.")

# Обработчик для изменения предпочтений в еде
@dp.message(UserForm.food_preferences)
async def process_food_preferences(message: types.Message, state: FSMContext):
    food_preferences = message.text.strip()
    await state.update_data(food_preferences=food_preferences)
    await message.answer(f"Предпочтения в еде обновлены: {food_preferences}")

    await state.set_state(None)

    await message.answer("Редактирование завершено. Выберите одну из команд:\n"
                         "/recommendation - Получить рекомендации по питанию\n"
                         "/train_recommendations - Получить рекомендации по тренировкам\n"
                         "/add_training - Добавить тренировку\n"
                         "/see_trainings - Посмотреть мои тренировки\n"
                         "/set_reminder_time - Настроить напоминания\n"
                         "/edit_profile - редактирование профиля.")


# Обработчик всех текстовых сообщений
@dp.message(F.text)
async def handle_text_message(message: Message, state: FSMContext):
    current_state = await state.get_state()

    if current_state is None:
        await message.answer(
            "Я не понимаю, что вы имеете в виду. Пожалуйста, используйте команды для взаимодействия с ботом.\n"
            "Доступные команды:\n"
            "/start - Начать взаимодействие\n"
            "/recommendation - Получить рекомендации по питанию\n"
            "/train_recommendations - Получить рекомендации по тренировкам\n"
            "/add_training - Добавить тренировку\n"
            "/see_trainings - Посмотреть мои тренировки\n"
            "/set_reminder_time - Настроить напоминания\n"
            "/edit_profile - редактирование профиля."
        )
    else:
        await message.answer("Пожалуйста, следуйте инструкциям для текущего действия.")


def register_handlers(dp: Dispatcher):
    dp.register_message_handler(cmd_start, Command("start"))
    dp.register_message_handler(process_age, state=UserForm.age)
    dp.register_message_handler(process_height, state=UserForm.height)
    dp.register_message_handler(process_weight, state=UserForm.weight)
    dp.register_message_handler(process_goals, state=UserForm.goals)
    dp.register_message_handler(process_food_preferences, state=UserForm.food_preferences)
    dp.register_message_handler(cmd_recommendations, Command("recommendation"))
    dp.register_message_handler(cmd_train_recommendations, Command("train_recommendations"))
    dp.register_message_handler(cmd_see_trainings, Command("see_trainings"))
    dp.register_message_handler(cmd_add_training, Command("add_training"))
    dp.register_message_handler(cmd_set_reminder_time, Command("set_reminder_time"))
    dp.register_message_handler(cmd_set_training_goal, Command("set_training_goal"))
    dp.register_message_handler(process_training, state=UserForm.adding_training)
    dp.register_message_handler(process_time_input, state=UserForm.waiting_for_time)
    dp.register_message_handler(process_reminder_frequency, state=UserForm.setting_reminder_frequency)
    dp.register_message_handler(select_reminder_type, state=UserForm.select_reminder_type)
    dp.register_message_handler(handle_text_message, state=None)

    dp.register_callback_query_handler(process_training_goal, F.data.startswith("training_goal:"))
    dp.register_callback_query_handler(process_new_goals, F.data.startswith('goal_'))
    dp.register_callback_query_handler(process_edit_choice, F.data.startswith("edit_"))


# Запуск бота
async def main():
    try:
      await init_db()
      print("Бот запущен...")
      await set_bot_commands()
      scheduler.start()
      await restore_jobs()
      await bot.delete_webhook(drop_pending_updates=True)
      await dp.start_polling(bot)


    finally:
      await bot.session.close()
      print("Бот остановлен")

if __name__ == '__main__':
    asyncio.run(main())

Бот запущен...


Бот остановлен
